<a href="https://colab.research.google.com/github/noahdanieldsouza/PAM-classification/blob/main/run_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchaudio numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 810.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install git+https://github.com/google-research/perch-hoplite.git

  Cloning https://github.com/google-research/perch-hoplite.git to /tmp/pip-req-build-u84c001q
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/perch-hoplite.git /tmp/pip-req-build-u84c001q
  Resolved https://github.com/google-research/perch-hoplite.git to commit 89510073db5052959ffd3589d2e773d9e7c3e5c8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.2 MB/s eta 0:00:00
 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Correct way to copy a folder:
!cp -r  /content/drive/MyDrive/classifier_training/8567.240831165948.wav /content

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#@title Load model and connect to database. { vertical-output: true }

#@markdown Location of database containing audio embeddings.
db_path = '/content/DB'  #@param {type:'string'}
#@markdown Identifier (eg, name) to attach to labels produced during validation.
annotator_id = 'linnaeus'  #@param {type:'string'}

db = sqlite_usearch_impl.SQLiteUsearchDB.create(db_path)
db_model_config = db.get_metadata('model_config')
embed_config = db.get_metadata('audio_sources')
model_class = model_configs.get_model_class(db_model_config.model_key)
embedding_model = model_class.from_config(db_model_config.model_config)
audio_sources = source_info.AudioSources.from_config_dict(embed_config)
if hasattr(embedding_model, 'window_size_s'):
  window_size_s = embedding_model.window_size_s
else:
  window_size_s = 5.0
audio_filepath_loader = audio_loader.make_filepath_loader(
    audio_sources=audio_sources,
    window_size_s=window_size_s,
    sample_rate_hz=embedding_model.sample_rate,
)

print("Embeddings in DB:", len(db.get_embedding_ids()))
print(db.get_classes())

In [ ]:


import os
import torch
import torchaudio
import numpy as np
import csv
import gc
import tempfile

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from perch_hoplite.agile import colab_utils, embed, source_info
from perch_hoplite.db import sqlite_usearch_impl
from perch_hoplite.zoo import model_configs
from perch_hoplite.agile.classifier import LinearClassifier

# --- Paths ---
db_path = '/content/drive/MyDrive/classifier_training/DB'
classifier_path = '/content/drive/MyDrive/classifier_training/DB/agile_classifier_v2.pt'
wav_path = '/content/drive/MyDrive/classifier_training/8567.240831165948.wav'
output_csv_path = '/content/chunk_classification_results.csv'

# --- Load DB + model ---
db = sqlite_usearch_impl.SQLiteUsearchDB.create(db_path)
db_model_config = db.get_metadata('model_config')
model_class = model_configs.get_model_class(db_model_config['model_key'])
embedding_model = model_class.from_config(db_model_config['model_config'])

# --- Load classifier ---
classifier = LinearClassifier.load(classifier_path)
class_names = classifier.classes
print("✅ Loaded classifier with classes:", class_names)

# --- Audio config ---
sample_rate = embedding_model.sample_rate
window_size_s = getattr(embedding_model, 'window_size_s', 5.0)
chunk_samples = int(window_size_s * sample_rate)

# --- Output CSV header ---
with open(output_csv_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['start_time', 'end_time', 'predicted_class', 'confidence'])

# --- Load audio info ---
info = torchaudio.info(wav_path)
total_frames = info.num_frames

# --- Process chunks ---
for i in range(0, total_frames, chunk_samples):
    if i + chunk_samples > total_frames:
        break
    print(f"🔄 Processing chunk: {i / sample_rate:.1f}s to {(i + chunk_samples) / sample_rate:.1f}s")

    # Load chunk
    chunk, sr = torchaudio.load(wav_path, frame_offset=i, num_frames=chunk_samples)
    if chunk.shape[0] > 1:
        chunk = chunk.mean(dim=0, keepdim=True)
    chunk_np = chunk.squeeze(0).numpy()

    try:
        # Get embeddings directly
        emb_outputs = embedding_model.embed(chunk_np)
        emb_vectors = emb_outputs.embeddings  # shape: (1, 1, D) or similar

        if emb_vectors is None or len(emb_vectors) == 0:
            raise ValueError("No embeddings returned")

        # Classify each embedding in the chunk
        for emb in emb_vectors:
            for emb_vec in emb:
                logits = classifier(emb_vec)
                probs = np.exp(logits) / np.sum(np.exp(logits))
                pred_idx = np.argmax(probs)
                pred_label = class_names[pred_idx]
                confidence = probs[pred_idx]

                start_time = i / sample_rate
                end_time = (i + chunk_samples) / sample_rate
                with open(output_csv_path, 'a', newline='') as f:
                    writer = csv.writer(f)
                    writer.writerow([start_time, end_time, pred_label, f"{confidence:.4f}"])

    except Exception as e:
        print(f"❌ Error processing chunk at {i/sample_rate:.1f}s: {e}")

    # Cleanup
    del chunk, chunk_np
    gc.collect()

print("✅ Classification complete. Results saved to:", output_csv_path)


Mounted at /content/drive
✅ Loaded classifier with classes: ['boat', 'non_boat']
🔄 Processing chunk: 0.0s to 5.0s
🔄 Processing chunk: 5.0s to 10.0s
🔄 Processing chunk: 10.0s to 15.0s
🔄 Processing chunk: 15.0s to 20.0s
🔄 Processing chunk: 20.0s to 25.0s
🔄 Processing chunk: 25.0s to 30.0s
🔄 Processing chunk: 30.0s to 35.0s
🔄 Processing chunk: 35.0s to 40.0s
🔄 Processing chunk: 40.0s to 45.0s
🔄 Processing chunk: 45.0s to 50.0s
🔄 Processing chunk: 50.0s to 55.0s
🔄 Processing chunk: 55.0s to 60.0s
🔄 Processing chunk: 60.0s to 65.0s
🔄 Processing chunk: 65.0s to 70.0s
🔄 Processing chunk: 70.0s to 75.0s
🔄 Processing chunk: 75.0s to 80.0s
🔄 Processing chunk: 80.0s to 85.0s
🔄 Processing chunk: 85.0s to 90.0s
🔄 Processing chunk: 90.0s to 95.0s
🔄 Processing chunk: 95.0s to 100.0s
🔄 Processing chunk: 100.0s to 105.0s
🔄 Processing chunk: 105.0s to 110.0s
🔄 Processing chunk: 110.0s to 115.0s
🔄 Processing chunk: 115.0s to 120.0s
🔄 Processing chunk: 120.0s to 125.0s
🔄 Processing chunk: 125.0s to 130.0s